First of all, make sure your environment has installed the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore) as well as other dependencies. Uncomment the following cell and run it.

In [1]:
#! pip install optimum[graphcore] sacrebleu

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/jincheng/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Then you need to install Git-LFS. Uncomment the following instructions:

In [3]:
# !apt install git-lfs

Let's print out the versions of Transformers and Optimum Graphcore:

In [4]:
import transformers
import optimum.graphcore

print(transformers.__version__)
print(optimum.graphcore.__version__)

4.21.1
0.3.1


# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](images/translation.png)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `IPUSeq2SeqTrainer` API.

In [5]:
model_checkpoint = "facebook/bart-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library and is supported by Optimum Graphcore. Here we picked the [`facebook/bart-base`](https://huggingface.co/facebook/bart-base) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [6]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Reusing dataset wmt16 (/localdata/jincheng/huggingface/datasets/wmt16/ro-en/1.0.0/28ebdf8cf22106c2f1e58b2083d4b103608acd7bfdb6b14313ccd9e5bc8c313a)


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

To access an actual element, you need to select a split first, then give an index:

In [8]:
raw_datasets["train"][0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [9]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'She has emphasised the correlation between reforms in the region and the countries' likelihood of accession to the EU.', 'ro': 'Dumneaei a subliniat corelaţia dintre reformele din această regiune şi probabilitatea de aderare a ţărilor la UE.'}"
1,"{'en': 'From the Community viewpoint, we are going to use EUR 30 000 million which will be, in practice, mobilised by the European Investment Bank, and in respect of the EUR 5 000 million which belongs, strictly speaking, to the Community budget, we are encountering major problems.', 'ro': 'La nivel comunitar, vom utiliza 30 de miliarde de euro, bani care vor fi practic mobilizaţi de Banca Europeană de Investiţii, iar în ceea ce priveşte deblocarea celor 5 miliarde de euro care provin, strict vorbind, de la bugetul comunitar, întâmpinăm probleme serioase.'}"
2,"{'en': 'We are a hi-tech location, we have a lot of opportunities, and researchers in Germany, for example, have called for a specialised monitoring network for those aerosols, those minuscule items, that volcanic ash contains.', 'ro': 'Suntem o regiune a tehnologiei înalte, avem numeroase oportunităţi, iar cercetătorii din Germania au solicitat, de exemplu, o reţea specializată de monitorizare pentru acei aerosoli, acele elemente minuscule, pe care le conţine cenuşa vulcanică.'}"
3,"{'en': 'We must use suitable means to try to simplify our business.', 'ro': 'Trebuie să utilizăm mijloace adecvate pentru a încerca să ne simplificăm afacerile.'}"
4,"{'en': 'The decline was mainly on the back of favourable exchange rate developments, which had a positive effect on dollar-denominated liabilities.', 'ro': 'Reducerea se datorează în principal evoluţiei favorabile a ratei de schimb, care a avut un efect pozitiv asupra obligaţiunilor exprimate în dolari.'}"


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [11]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [12]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [13]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

For the mBART tokenizer, we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [14]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [15]:
tokenizer("Hello, this one sentence!")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [16]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [17]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[0, 31414, 6, 42, 65, 3645, 328, 2], [0, 713, 16, 277, 3645, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [18]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed the `tokenizer` with three arguments. `padding="max_length"` will ensure that an input shorter than maximum length will be padded to the maximum length. `truncation=True` will ensure that an input longer than maximum length will be truncated to the maximum length. `max_length=max_input/target_length` sets the maximum length of a sequence.

Note that it is necessary to pad all the sentences to the same length since currently Graphcore's PyTorch implementation only runs in static mode.

In [19]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding="max_length", truncation=True)
        
    # Since we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [20]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[0, 31339, 4128, 9, 3879, 35, 192, 18925, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 19186, 1001, 6486, 9, 18925, 9, 986, 2828, 35, 192, 18925, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [21]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wmt16/ro-en/1.0.0/28ebdf8cf22106c2f1e58b2083d4b103608acd7bfdb6b14313ccd9e5bc8c313a/cache-b8ff349bb1c08367.arrow
Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wmt16/ro-en/1.0.0/28ebdf8cf22106c2f1e58b2083d4b103608acd7bfdb6b14313ccd9e5bc8c313a/cache-f7b7beb6d9b88b0a.arrow
Loading cached processed dataset at /localdata/jincheng/huggingface/datasets/wmt16/ro-en/1.0.0/28ebdf8cf22106c2f1e58b2083d4b103608acd7bfdb6b14313ccd9e5bc8c313a/cache-ea057d8fb57c37f4.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [22]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from optimum.graphcore import IPUConfig, IPUSeq2SeqTrainer, IPUSeq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `IPUSeq2SeqTrainer`, we will need to define four more things. The first thing we need to define is the `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with one config name or path:

In [23]:
ipu_config_name = 'Graphcore/bart-base-ipu'
ipu_config = IPUConfig.from_pretrained(
    ipu_config_name,
)

The other thing we need to define is the `IPUSeq2SeqTrainingArguments`, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [24]:
micro_batch_size = 1
gradient_accumulation_steps = 128
pod_type = "pod16"

model_name = model_checkpoint.split("/")[-1]
args = IPUSeq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    pod_type=pod_type,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
#     predict_with_generate=True,
    dataloader_drop_last=True,
    logging_steps=20,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the three batch-size-related arguments, namely `micro_batch_size`, `gradient_accumulation_steps` and `pod_type` defined at the top of the cell and customize the weight decay. Since the `IPUSeq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Note that the `predict_with_generate` option is disabled since it is not supported yet.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/marian-finetuned-en-to-ro"` or `"huggingface/marian-finetuned-en-to-ro"`).

Then, we need a special kind of data collator, which will prepare the `decoder_input_ids`:

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `IPUSeq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [26]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `IPUSeq2SeqTrainer`:

In [27]:
trainer = IPUSeq2SeqTrainer(
    model,
    ipu_config,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
#     compute_metrics=compute_metrics
    compute_metrics=None # Use None since predict_with_generate is not supported yet.
)

Overriding IPU config: gradient_accumulation_steps=128
-------------------- Device Allocation --------------------
Embedding  --> IPU 0
Encoder 0  --> IPU 1
Encoder 1  --> IPU 1
Encoder 2  --> IPU 1
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Decoder 0  --> IPU 2
Decoder 1  --> IPU 2
Decoder 2  --> IPU 2
Decoder 3  --> IPU 3
Decoder 4  --> IPU 3
Decoder 5  --> IPU 3
LM Head Output --> IPU 0
-----------------------------------------------------------
/localdata/jincheng/optimum-graphcore/notebooks/bart-base-finetuned-en-to-ro is already a clone of https://huggingface.co/Jinchen/bart-base-finetuned-en-to-ro. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `PipelinedBartForConditionalGeneration.forward` and have been ignored: translation.
Compiling Model...
/localdata/jincheng/sdks/poplar_sdk-ubuntu_20_04-2.6.0+1066-459b49240b/2.6.0+1066_poptorch/lib/python3.8/site-packages/optimum/graphcore/models/bart/modeling_bart.py:143: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/localdata/jincheng/sdks/poplar_sdk-ubuntu_20_04-2.6.0+1066-459b49240b/2.6.0+1066_poptorch/lib/python3.8/site-packages/optimum/graphcore/models/bart/modeling_bart.py:176: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python valu

  0%|          | 0/1192 [00:00<?, ?it/s]

{'loss': 4.3099, 'learning_rate': 1.9664429530201344e-05, 'epoch': 0.02}
{'loss': 2.9674, 'learning_rate': 1.9328859060402687e-05, 'epoch': 0.03}
{'loss': 2.6965, 'learning_rate': 1.899328859060403e-05, 'epoch': 0.05}
{'loss': 2.5128, 'learning_rate': 1.865771812080537e-05, 'epoch': 0.07}
{'loss': 2.5123, 'learning_rate': 1.832214765100671e-05, 'epoch': 0.08}
{'loss': 2.4653, 'learning_rate': 1.7986577181208054e-05, 'epoch': 0.1}
{'loss': 2.3489, 'learning_rate': 1.7651006711409397e-05, 'epoch': 0.12}
{'loss': 2.3844, 'learning_rate': 1.731543624161074e-05, 'epoch': 0.13}
{'loss': 2.112, 'learning_rate': 1.6979865771812082e-05, 'epoch': 0.15}
{'loss': 2.2164, 'learning_rate': 1.6644295302013425e-05, 'epoch': 0.17}
{'loss': 2.104, 'learning_rate': 1.6308724832214767e-05, 'epoch': 0.18}
{'loss': 2.1377, 'learning_rate': 1.5973154362416107e-05, 'epoch': 0.2}
{'loss': 2.0717, 'learning_rate': 1.5637583892617453e-05, 'epoch': 0.22}
{'loss': 1.9549, 'learning_rate': 1.5302013422818792e-05, '

Saving model checkpoint to bart-base-finetuned-en-to-ro/checkpoint-500


{'loss': 1.7908, 'learning_rate': 1.1610738255033558e-05, 'epoch': 0.42}


...

  0%|          | 0/83 [00:00<?, ?it/s]

{'eval_loss': 1.9912109375, 'eval_runtime': 4.0909, 'eval_samples_per_second': 486.93, 'eval_steps_per_second': 20.289, 'epoch': 1.0}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 1046.6743, 'train_samples_per_second': 583.089, 'train_steps_per_second': 1.139, 'train_loss': 1.8398826650325084, 'epoch': 1.0}


TrainOutput(global_step=1192, training_loss=1.8398826650325084, metrics={'train_runtime': 1046.6743, 'train_samples_per_second': 583.089, 'train_steps_per_second': 1.139, 'train_loss': 1.8398826650325084, 'epoch': 1.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [29]:
trainer.push_to_hub()

Saving model checkpoint to bart-base-finetuned-en-to-ro
-------------------- Device Allocation --------------------
Embedding  --> IPU 0
Encoder 0  --> IPU 1
Encoder 1  --> IPU 1
Encoder 2  --> IPU 1
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Decoder 0  --> IPU 2
Decoder 1  --> IPU 2
Decoder 2  --> IPU 2
Decoder 3  --> IPU 3
Decoder 4  --> IPU 3
Decoder 5  --> IPU 3
LM Head Output --> IPU 0
-----------------------------------------------------------
Configuration saved in bart-base-finetuned-en-to-ro/ipu_config.json


Upload file pytorch_model.bin:   0%|          | 32.0k/266M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Jinchen/bart-base-finetuned-en-to-ro
   5ece51b..5466ebd  main -> main

To https://huggingface.co/Jinchen/bart-base-finetuned-en-to-ro
   5466ebd..b3aec0a  main -> main



'https://huggingface.co/Jinchen/bart-base-finetuned-en-to-ro/commit/5466ebd34de529e09ff822282332adb073127a12'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```